<a href="https://colab.research.google.com/github/DimasAsyura/Data-Mining/blob/main/preprocessing_data_mining_dimas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns



column_names = [
    'mpg', 'cylinders', 'displacement', 'horsepower', 'weight',
    'acceleration', 'model_year', 'origin', 'car_name'
]


print("--- Memuat Data ---")
try:
    mpg_df = pd.read_csv(
        'auto-mpg.data',
        header=None,
        names=column_names,
        na_values='?',
        delim_whitespace=True
    )
    print("Dataset 'auto-mpg.data' berhasil dimuat.")
    print(f"Bentuk dataset awal: {mpg_df.shape}")
    print("\n")

except FileNotFoundError:
    print("ERROR: File 'auto-mpg.data' tidak ditemukan. Pastikan file berada di direktori yang sama.")
    exit()
except Exception as e:
    print(f"ERROR saat memuat data: {e}")
    exit()


print("--- Analisis Data Eksploratif Awal ---")
print("Informasi umum dataset:")
mpg_df.info()
print("\n")

print("Ringkasan statistik deskriptif:")
print(mpg_df.describe())
print("\n")

print("Jumlah nilai yang hilang di setiap kolom SEBELUM preprocessing:")
print(mpg_df.isnull().sum())
print("\n" + "="*70 + "\n")




print("--- Memulai Tahap Preprocessing Data ---")



if mpg_df['horsepower'].isnull().any():
    print("Menangani nilai hilang di kolom 'horsepower'...")

    median_hp = mpg_df['horsepower'].median()
    mpg_df['horsepower'].fillna(median_hp, inplace=True)
    print(f"Nilai hilang 'horsepower' diisi dengan median: {median_hp}")
else:
    print("Tidak ada nilai hilang di kolom 'horsepower'.")

print("\nJumlah nilai yang hilang di setiap kolom SETELAH penanganan:")
print(mpg_df.isnull().sum())
print("\n")




if 'car_name' in mpg_df.columns:
    print("Menghapus kolom 'car_name' (non-numerik dan unik)...")
    mpg_df_processed = mpg_df.drop(columns=['car_name']).copy()
    print("Kolom 'car_name' berhasil dihapus.")
else:
    mpg_df_processed = mpg_df.copy()
    print("Kolom 'car_name' tidak ditemukan atau sudah dihapus.")
print(f"Bentuk dataset setelah menghapus 'car_name': {mpg_df_processed.shape}")
print("\n")




print("Mengubah kolom 'origin' menjadi variabel dummy (One-Hot Encoding)...")

mpg_df_encoded = pd.get_dummies(mpg_df_processed, columns=['origin'], prefix='origin', drop_first=True)

print("Kolom 'origin' berhasil di-encode.")
print(f"Bentuk dataset setelah One-Hot Encoding 'origin': {mpg_df_encoded.shape}")
print("Contoh 5 baris pertama setelah encoding:")
print(mpg_df_encoded.head())
print("\n")


print("Melakukan standardisasi (MinMaxScaler) pada fitur numerik...")
from sklearn.preprocessing import MinMaxScaler


features_to_scale = ['cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'model_year']

scaler = MinMaxScaler()
mpg_df_scaled_features = mpg_df_encoded.copy()
mpg_df_scaled_features[features_to_scale] = scaler.fit_transform(mpg_df_encoded[features_to_scale])

print("Fitur numerik berhasil distandardisasi.")
print("Contoh 5 baris pertama setelah standardisasi (fitur tertentu):")
print(mpg_df_scaled_features.head())
print("\n" + "="*70 + "\n")



print("--- Verifikasi Hasil Preprocessing ---")
print("Informasi dataset setelah semua preprocessing:")
mpg_df_scaled_features.info()
print("\n")

print("Ringkasan statistik deskriptif setelah standardisasi:")
print(mpg_df_scaled_features.describe())
print("\n")

print("Dataset akhir yang sudah diproses siap untuk analisis/pemodelan:")
print(mpg_df_scaled_features.head())



plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
sns.histplot(mpg_df['horsepower'], kde=True)
plt.title('Distribusi Horsepower (Setelah Imputasi Median)')
plt.xlabel('Horsepower')
plt.ylabel('Frekuensi')


plt.subplot(1, 2, 2)
sns.histplot(mpg_df_processed['weight'], kde=True, color='red', label='Sebelum Scaling')
sns.histplot(mpg_df_scaled_features['weight'], kde=True, color='blue', label='Setelah Scaling')
plt.title('Distribusi Weight (Sebelum & Sesudah Scaling)')
plt.xlabel('Weight')
plt.ylabel('Frekuensi')
plt.legend()
plt.tight_layout()
plt.savefig('preprocessing_verification_plots.png')
print("\nVisualisasi verifikasi preprocessing telah disimpan sebagai 'preprocessing_verification_plots.png'.")

print("\nProses preprocessing selesai!")